In [55]:
#Generates a set of coefficients as a sage "vector"
def coefs_1d(N,N0,lab) :
    return vector([ var(lab+'%s'%i) for i in range(N0,N0+N) ])


#Generates a 1-D polynomial as a sage expreession based on a set of coefficients
def poly_1d(N,coefs,x) :
    return sum( vector([ coefs[i]*x^i for i in range(N) ]) )

############################################################################
## Computes Gauss-Lobatto quadrature weights and nodes to integrate over
## the domain [-1,1]. Adapted from matlab algorithm in
## http://www.mathworks.com/matlabcentral/fileexchange/4775-legende-gauss-lobatto-nodes-and-weights
############################################################################
## nnodes  : Total number of integration points
## prec    : Floating Point Precision of the result
## verbose : Print things as you iterations proceed? (True or False)
## tol     : Tolerance for L-infinity of difference in old and new guesses
## mxcount : Maximum number of allowed iterations
############################################################################
def lobatto_weights_nodes(nnodes,prec,verbose,tol,mxcount) :
    N1 = nnodes
    N = N1-1
    x = vector( [ cos(pi*(i)/(N)+pi) for i in range(N1) ] )
    x = x.n(prec)
    P = matrix( N1 , N1 , [ 0.n(prec) for i in range(N1^2)] )
    xold = x * 2
    linf = max(x-xold)/max(x)
    count = 0
    while ( linf > tol and count < mxcount ) :
        xold = x
        P[:,0] = 1
        P[:,1] = x
        for k in range(1,N) :
            tmp = matrix(N1,1, [ x[i] * P[i,k] for i in range(N1) ] )
            P[:,k+1] = ( (2*(k+1)-1)*tmp - ((k+1)-1)*P[:,k-1] )/(k+1)
        tmp  = matrix(N1,1, [ x[i] * P[i,N] - P[i,N-1] for i in range(N1) ] )
        tmp2 = matrix(N1,1, [ N1*P[i,N] for i in range(N1) ] )
        tmp3 = vector( [ tmp[i,0] / tmp2[i,0] for i in range(N1) ] )
        x = xold - tmp3
        linf = max(x-xold)/max(x)
        count = count + 1
        if (verbose) :
            print(count,linf)
    w = vector( [ 2 / (N*N1*P[i,N]^2) for i in range(N1) ] )
    return x,w


def tanh_cluster(N,mu) :
    return vector( [ (tanh((2*mu*i)/(N-1)-mu)/tanh(mu)+1)/2-1/2 for i in range(N) ] )


#Returns N GLL point locations in [-0.5,0.5] (quad precision)
def points_gll(N) :
    x,w = lobatto_weights_nodes(N,129,False,1e-35,20)
    return vector([ x[i]/2 for i in range(N) ])

def c_scalar(retlab,val,coeflab) :
    import re
    code = ""
    s = retlab+" = "
    #Remove spaces
    s = s+str(val).replace(' ','')
    #Replace squares with multiplication
    s = re.sub("([a-zA-Z()0-9_]*)\\^2","(\\1*\\1)",s,0,re.DOTALL)
    #Replace exponentials with power functions
    s = re.sub("([a-zA-Z()0-9_]*)\\^([0-9]*)","pow((double)\\1,(double)\\2)",s,0,re.DOTALL)
    #Replace coeficients (e.g., c1) with parentheses (e.g., c(1))
    s = re.sub(coeflab+"([0-9]+)",coeflab+"(\\1)",s, 0, re.DOTALL)
    #Add _fp suffixes to floating point numbers
    s = re.sub("([0-9]*\\.[0-9e-]*)","static_cast<real>(\\1)",s, 0, re.DOTALL)
    #Add new line to the end.
    code = code + s + ";\n"
    return code

In [56]:
# Left
coefs = coefs_1d(2,0,'a')
p = poly_1d(2,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pL = poly_1d(2,constr_to_coefs*coefs_1d(2,0,'s'),x)

# Right
coefs = coefs_1d(2,0,'a')
p = poly_1d(2,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pR = poly_1d(2,constr_to_coefs*coefs_1d(2,1,'s'),x)

# High
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(3,constr_to_coefs*coefs_1d(3,0,'s'),x)

var('gamma_L,gamma_R')
pts = points_gll(4)
for i in range(len(pts)) :
    xloc = pts[i]
    sten = coefs_1d(3,0,'s')
    stencil_weights = jacobian(pL.subs(x=xloc)*gamma_L+pR.subs(x=xloc)*gamma_R,sten)
    A = jacobian(stencil_weights,[gamma_L,gamma_R])
    rhs = Matrix(3,1,jacobian(pH.subs(x=xloc),sten).list())
    print((A.transpose()*A)^-1*A.transpose()*rhs)
    var('w_L,w_R')
    print(c_scalar( f'q{i+1}' , w_L*pL.subs(x=xloc) + w_R*pR.subs(x=xloc) , 's' ))

[0.66666666666666666666666666666666666667]
[0.33333333333333333333333333333333333333]
q1 = (static_cast<real>(0.50000000000000000000000000000000000000)*s(0)+static_cast<real>(0.50000000000000000000000000000000000000)*s(1))*w_L+(static_cast<real>(1.5000000000000000000000000000000000000)*s(1)-static_cast<real>(0.50000000000000000000000000000000000000)*s(2))*w_R;

[0.42546440075000701011969421104229079215]
[0.57453559924999298988030578895770920783]
q2 = (static_cast<real>(0.22360679774997896964091736687312762354)*s(0)+static_cast<real>(0.77639320225002103035908263312687237646)*s(1))*w_L+(static_cast<real>(1.2236067977499789696409173668731276235)*s(1)-static_cast<real>(0.22360679774997896964091736687312762354)*s(2))*w_R;

[0.57453559924999298988030578895770920785]
[0.42546440075000701011969421104229079215]
q3 = (-static_cast<real>(0.22360679774997896964091736687312762354)*s(0)+static_cast<real>(1.2236067977499789696409173668731276235)*s(1))*w_L+(static_cast<real>(0.776393202250021030359082

In [57]:
# Left
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pL = poly_1d(3,constr_to_coefs*coefs_1d(3,0,'s'),x)

# Center
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pC = poly_1d(3,constr_to_coefs*coefs_1d(3,1,'s'),x)

# Right
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pR = poly_1d(3,constr_to_coefs*coefs_1d(3,2,'s'),x)

# High
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(5,constr_to_coefs*coefs_1d(5,0,'s'),x)

var('gamma_L,gamma_C,gamma_R')
pts = points_gll(4)
for i in range(len(pts)) :
    xloc = pts[i]
    sten = coefs_1d(5,0,'s')
    stencil_weights = jacobian(pL.subs(x=xloc)*gamma_L+
                               pC.subs(x=xloc)*gamma_C+
                               pR.subs(x=xloc)*gamma_R,sten)
    A = jacobian(stencil_weights,[gamma_L,gamma_C,gamma_R])
    rhs = Matrix(5,1,jacobian(pH.subs(x=xloc),sten).list())
    print((A.transpose()*A)^-1*A.transpose()*rhs)
    var('w_L,w_C,w_R')
    print(c_scalar( f'q{i+1}' , w_L*pL.subs(x=xloc) + w_C*pC.subs(x=xloc) + w_R*pR.subs(x=xloc) , 's' ))

[0.30000000000000000000000000000000000001]
[0.60000000000000000000000000000000000001]
[0.10000000000000000000000000000000000000]
q1 = (static_cast<real>(0.33333333333333333333333333333333333333)*s(1)+static_cast<real>(0.83333333333333333333333333333333333333)*s(2)-static_cast<real>(0.16666666666666666666666666666666666667)*s(3))*w_C+(-static_cast<real>(0.16666666666666666666666666666666666667)*s(0)+static_cast<real>(0.83333333333333333333333333333333333333)*s(1)+static_cast<real>(0.33333333333333333333333333333333333333)*s(2))*w_L+(static_cast<real>(1.8333333333333333333333333333333333333)*s(2)-static_cast<real>(1.1666666666666666666666666666666666667)*s(3)+static_cast<real>(0.33333333333333333333333333333333333333)*s(4))*w_R;

[0.16108042773295884711890326586686025883]
[0.58636363636363636363636363636363636378]
[0.25255593590340478924473309776950337754]
q2 = (static_cast<real>(0.095136732208322818153792016769897145105)*s(1)+static_cast<real>(1.0333333333333333333333333333333333333)*s(

In [59]:
# 1
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p1 = poly_1d(4,constr_to_coefs*coefs_1d(4,0,'s'),x)

# 2
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p2 = poly_1d(4,constr_to_coefs*coefs_1d(4,1,'s'),x)

# 3
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p3 = poly_1d(4,constr_to_coefs*coefs_1d(4,2,'s'),x)

# 4
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p4 = poly_1d(4,constr_to_coefs*coefs_1d(4,3,'s'),x)

# High
coefs = coefs_1d(7,0,'a')
p = poly_1d(7,coefs,x)
constr = vector([ integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(7,constr_to_coefs*coefs_1d(7,0,'s'),x)

var('gamma_1,gamma_2,gamma_3,gamma_4')
pts = points_gll(4)
for i in range(len(pts)) :
    xloc = pts[i]
    stencil_weights = jacobian(p1.subs(x=xloc)*gamma_1+
                               p2.subs(x=xloc)*gamma_2+
                               p3.subs(x=xloc)*gamma_3+
                               p4.subs(x=xloc)*gamma_4,coefs_1d(7,0,'s'))
    A = jacobian(stencil_weights,[gamma_1,gamma_2,gamma_3,gamma_4])
    sten = coefs_1d(7,0,'s')
    rhs = Matrix(7,1,jacobian(pH.subs(x=xloc),sten).list())
    print((A.transpose()*A)^-1*A.transpose()*rhs)
    var('w_1,w_2,w_3,w_4')
    print(c_scalar( f'q{i+1}' , w_1*p1.subs(x=xloc) + w_2*p2.subs(x=xloc) + w_3*p3.subs(x=xloc) + w_4*p4.subs(x=xloc) , 's' ))

[ 0.11428571428571428571428571428571428574]
[ 0.51428571428571428571428571428571428564]
[ 0.34285714285714285714285714285714285724]
[0.028571428571428571428571428571428571427]
q1 = (static_cast<real>(0.083333333333333333333333333333333333333)*s(0)-static_cast<real>(0.41666666666666666666666666666666666667)*s(1)+static_cast<real>(1.0833333333333333333333333333333333333)*s(2)+static_cast<real>(0.25000000000000000000000000000000000000)*s(3))*w_1+(-static_cast<real>(0.083333333333333333333333333333333333333)*s(1)+static_cast<real>(0.58333333333333333333333333333333333333)*s(2)+static_cast<real>(0.58333333333333333333333333333333333333)*s(3)-static_cast<real>(0.083333333333333333333333333333333333333)*s(4))*w_2+(static_cast<real>(0.25000000000000000000000000000000000000)*s(2)+static_cast<real>(1.0833333333333333333333333333333333333)*s(3)-static_cast<real>(0.41666666666666666666666666666666666667)*s(4)+static_cast<real>(0.083333333333333333333333333333333333333)*s(5))*w_3+(static_cast<real>

In [62]:
# 1
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-9/2,-7/2) ,
                  integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p1 = poly_1d(5,constr_to_coefs*coefs_1d(5,0,'s'),x)

# 2
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p2 = poly_1d(5,constr_to_coefs*coefs_1d(5,1,'s'),x)

# 3
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p3 = poly_1d(5,constr_to_coefs*coefs_1d(5,2,'s'),x)

# 4
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p4 = poly_1d(5,constr_to_coefs*coefs_1d(5,3,'s'),x)

# 5
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ,
                  integrate(p,x, 7/2, 9/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p5 = poly_1d(5,constr_to_coefs*coefs_1d(5,4,'s'),x)

# High
coefs = coefs_1d(9,0,'a')
p = poly_1d(9,coefs,x)
constr = vector([ integrate(p,x,-9/2,-7/2) ,
                  integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ,
                  integrate(p,x, 7/2, 9/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(9,constr_to_coefs*coefs_1d(9,0,'s'),x)

var('gamma_1,gamma_2,gamma_3,gamma_4,gamma_5')
pts = points_gll(4)
for i in range(len(pts)) :
    xloc = pts[i]
    stencil_weights = jacobian(p1.subs(x=xloc)*gamma_1+
                               p2.subs(x=xloc)*gamma_2+
                               p3.subs(x=xloc)*gamma_3+
                               p4.subs(x=xloc)*gamma_4+
                               p5.subs(x=xloc)*gamma_5,coefs_1d(9,0,'s'))
    A = jacobian(stencil_weights,[gamma_1,gamma_2,gamma_3,gamma_4,gamma_5])
    sten = coefs_1d(9,0,'s')
    rhs = Matrix(9,1,jacobian(pH.subs(x=xloc),sten).list())
    print((A.transpose()*A)^-1*A.transpose()*rhs)
    var('w_1,w_2,w_3,w_4,w_5')
    print(c_scalar( f'q{i+1}' , w_1*p1.subs(x=xloc) + w_2*p2.subs(x=xloc) + w_3*p3.subs(x=xloc) + w_4*p4.subs(x=xloc) + w_5*p5.subs(x=xloc) , 's' ))

[ 0.039682539682539682539682539682539682577]
[  0.31746031746031746031746031746031746117]
[  0.47619047619047619047619047619047618949]
[  0.15873015873015873015873015873015872971]
[0.0079365079365079365079365079365079365201]
q1 = (-static_cast<real>(0.050000000000000000000000000000000000000)*s(0)+static_cast<real>(0.28333333333333333333333333333333333333)*s(1)-static_cast<real>(0.71666666666666666666666666666666666667)*s(2)+static_cast<real>(1.2833333333333333333333333333333333333)*s(3)+static_cast<real>(0.20000000000000000000000000000000000000)*s(4))*w_1+(static_cast<real>(0.033333333333333333333333333333333333333)*s(1)-static_cast<real>(0.21666666666666666666666666666666666667)*s(2)+static_cast<real>(0.78333333333333333333333333333333333333)*s(3)+static_cast<real>(0.45000000000000000000000000000000000000)*s(4)-static_cast<real>(0.050000000000000000000000000000000000000)*s(5))*w_2+(-static_cast<real>(0.050000000000000000000000000000000000000)*s(2)+static_cast<real>(0.45000000000000000

In [63]:
print(pts)

(-0.50000000000000000000000000000000000000, -0.22360679774997896964091736687312762354, 0.22360679774997896964091736687312762354, 0.50000000000000000000000000000000000000)
